# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    nhl2018 = nhl_df.loc[nhl_df['year'] == 2018, ['team', 'W', 'L']]
    nhl2018 = nhl2018[nhl2018['W'].str.isdigit()]
    nhl2018['team'] = nhl2018['team'].replace(regex=r'\*$', value='')
    nhl2018['team_short'] = [team.split(' ')[-1] for team in nhl2018['team']]
    
    cities_nhl = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
    cities_nhl.rename(columns={'Metropolitan area': 'Metropolitan', 'Population (2016 est.)[8]': 'Population'}, 
                      inplace=True)
    cities_nhl['NHL'] = cities_nhl['NHL'].replace(regex=r'\[.*\]', value='').replace({'—': np.nan, '': np.nan})
    cities_nhl.dropna(inplace=True)
    cities_nhl['NHL'] = [re.findall(r'[A-Z][^A-Z]*', re.sub(r'\s', '', team)) for team in cities_nhl['NHL']]
    
    new_df = []
    for i in cities_nhl.index:
        for team in cities_nhl['NHL'][i]:
            new_df.append([cities_nhl['Metropolitan'][i], cities_nhl['Population'][i], team])
    cities_nhl_full = pd.DataFrame(new_df, columns=cities_nhl.columns)
    
    nhl = nhl2018.merge(cities_nhl_full, left_on='team_short', right_on='NHL')[['Metropolitan', 'Population', 'W', 'L']]
    nhl = nhl.astype({'Population': 'int', 'W': 'int', 'L': 'int'})
    nhl['WinLoss'] = nhl['W'] / (nhl['W']+nhl['L'])
    nhl = nhl.groupby(['Metropolitan', 'Population']).agg('mean').reset_index()
    #nhl['WinLoss'] = nhl['W'] / nhl['L']
    
    population_by_region = [pop for pop in nhl['Population']] # pass in metropolitan area population from cities
    win_loss_by_region = [wl for wl in nhl['WinLoss']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    nba2018 = nba_df.loc[nba_df['year'] == 2018, ['team', 'W', 'L']]
    nba2018['team'] = nba2018['team'].replace(regex=r'[\*]*\s\(\d+\)$', value='')
    nba2018['team_short'] = [team.split(' ')[-1] for team in nba2018['team']]
    
    cities_nba = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']]
    cities_nba.rename(columns={'Metropolitan area': 'Metropolitan', 'Population (2016 est.)[8]': 'Population'}, 
                      inplace=True)
    cities_nba['NBA'] = cities_nba['NBA'].replace(regex=r'\[.*\]', value='').replace({'—': np.nan, '': np.nan})
    cities_nba.dropna(inplace=True)
    cities_nba['NBA'] = [re.findall(r'[A-Z0-9][^A-Z]*', re.sub(r'\s', '', team)) for team in cities_nba['NBA']]
    
    new_df = []
    for i in cities_nba.index:
        for team in cities_nba['NBA'][i]:
            new_df.append([cities_nba['Metropolitan'][i], cities_nba['Population'][i], team])
    cities_nba_full = pd.DataFrame(new_df, columns=cities_nba.columns)
    
    nba = nba2018.merge(cities_nba_full, left_on='team_short', right_on='NBA')[['Metropolitan', 'Population', 'W', 'L']]
    nba = nba.astype({'Population': 'int', 'W': 'int', 'L': 'int'})
    nba['WinLoss'] = nba['W'] / (nba['W']+nba['L'])
    nba = nba.groupby(['Metropolitan', 'Population']).agg('mean').reset_index()
    #nba['WinLoss'] = nba['W'] / nba['L']
    
    population_by_region = [pop for pop in nba['Population']] # pass in metropolitan area population from cities
    win_loss_by_region = [wl for wl in nba['WinLoss']] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [22]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    mlb2018 = mlb_df.loc[mlb_df['year'] == 2018, ['team', 'W', 'L']]
    mlb2018['team_short'] = [team.split(' ')[-1] for team in mlb2018['team']]
    mlb2018['team_short'][mlb2018['team_short'].duplicated()] += '1'
    
    cities_mlb = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']]
    cities_mlb.rename(columns={'Metropolitan area': 'Metropolitan', 'Population (2016 est.)[8]': 'Population'}, 
                      inplace=True)
    cities_mlb['MLB'] = cities_mlb['MLB'].replace(regex=r'\[.*\]', value='').replace({'—': np.nan, '': np.nan})
    cities_mlb.dropna(inplace=True)
    cities_mlb['MLB'] = [re.findall(r'[A-Z][^A-Z]*', re.sub(r'\s', '', team)) for team in cities_mlb['MLB']]
    cities_mlb['MLB'][cities_mlb['Metropolitan'] == 'Chicago'][3][2] = 'Sox1'
    
    new_df = []
    for i in cities_mlb.index:
        for team in cities_mlb['MLB'][i]:
            new_df.append([cities_mlb['Metropolitan'][i], cities_mlb['Population'][i], team])
    cities_mlb_full = pd.DataFrame(new_df, columns=cities_mlb.columns)
    
    mlb = mlb2018.merge(cities_mlb_full, left_on='team_short', right_on='MLB')[['Metropolitan', 'Population', 'W', 'L']]
    mlb = mlb.astype({'Population': 'int', 'W': 'int', 'L': 'int'})
    mlb['WinLoss'] = mlb['W'] / (mlb['W']+mlb['L'])
    mlb = mlb.groupby(['Metropolitan', 'Population']).agg('mean').reset_index()
    #mlb['WinLoss'] = mlb['W'] / mlb['L']
    
    
    population_by_region = [pop for pop in mlb['Population']] # pass in metropolitan area population from cities
    win_loss_by_region = [wl for wl in mlb['WinLoss']] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [29]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    nfl2018 = nfl_df.loc[nfl_df['year'] == 2018, ['team', 'W', 'L']]
    nfl2018 = nfl2018[nfl2018['W'].str.isdigit()]
    nfl2018['team'] = nfl2018['team'].replace(regex=r'[\*\+]*$', value='')
    nfl2018['team_short'] = [team.split(' ')[-1] for team in nfl2018['team']]
    
    cities_nfl = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
    cities_nfl.rename(columns={'Metropolitan area': 'Metropolitan', 'Population (2016 est.)[8]': 'Population'}, 
                      inplace=True)
    cities_nfl['NFL'] = cities_nfl['NFL'].replace(regex=r'\[.*\]', value='').replace({'—': np.nan, '': np.nan})
    cities_nfl.dropna(inplace=True)
    cities_nfl['NFL'] = [re.findall(r'[A-Z0-9][^A-Z]*', re.sub(r'\s', '', team)) for team in cities_nfl['NFL']]
    
    new_df = []
    for i in cities_nfl.index:
        for team in cities_nfl['NFL'][i]:
            new_df.append([cities_nfl['Metropolitan'][i], cities_nfl['Population'][i], team])
    cities_nfl_full = pd.DataFrame(new_df, columns=cities_nfl.columns)
    
    nfl = nfl2018.merge(cities_nfl_full, left_on='team_short', right_on='NFL')[['Metropolitan', 'Population', 'W', 'L']]
    nfl = nfl.astype({'Population': 'int', 'W': 'int', 'L': 'int'})
    nfl['WinLoss'] = nfl['W'] / (nfl['W']+nfl['L'])
    nfl = nfl.groupby(['Metropolitan', 'Population']).agg('mean').reset_index()
    
    
    population_by_region = [pop for pop in nfl['Population'] ] # pass in metropolitan area population from cities
    win_loss_by_region = [wl for wl in nfl['WinLoss']] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [75]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    cities.rename(columns={'Metropolitan area': 'Metropolitan'}, inplace=True)
    
    
    
    nhl2018 = nhl_df.loc[nhl_df['year'] == 2018, ['team', 'W', 'L']]
    nhl2018 = nhl2018[nhl2018['W'].str.isdigit()]
    nhl2018['team'] = nhl2018['team'].replace(regex=r'\*$', value='')
    nhl2018['team_short'] = [team.split(' ')[-1] for team in nhl2018['team']]
    nhl2018 = nhl2018.astype({'W': 'int', 'L': 'int'})
    nhl2018 = nhl2018[['team_short', 'W', 'L']]
    
    cities_nhl = cities[['Metropolitan', 'NHL']]
    cities_nhl['NHL'] = cities_nhl['NHL'].replace(regex=r'\[.*\]', value='').replace({'—': np.nan, '': np.nan})
    cities_nhl.dropna(inplace=True)
    cities_nhl['NHL'] = [re.findall(r'[A-Z][^A-Z]*', re.sub(r'\s', '', team)) for team in cities_nhl['NHL']]
    
    new_df = []
    for i in cities_nhl.index:
        for team in cities_nhl['NHL'][i]:
            new_df.append([cities_nhl['Metropolitan'][i], team])
    cities_nhl_full = pd.DataFrame(new_df, columns=cities_nhl.columns)
    
    nhl = nhl2018.merge(cities_nhl_full, left_on='team_short', right_on='NHL')[['Metropolitan', 'W', 'L']]
    nhl['WinLoss_NHL'] = nhl['W'] / (nhl['W']+nhl['L'])
    nhl = nhl.groupby('Metropolitan').agg('mean').reset_index()
    
    
    
    nba2018 = nba_df.loc[nba_df['year'] == 2018, ['team', 'W', 'L']]
    nba2018['team'] = nba2018['team'].replace(regex=r'[\*]*\s\(\d+\)$', value='')
    nba2018['team_short'] = [team.split(' ')[-1] for team in nba2018['team']]
    nba2018 = nba2018.astype({'W': 'int', 'L': 'int'})
    nba2018 = nba2018[['team_short', 'W', 'L']]
    
    cities_nba = cities[['Metropolitan', 'NBA']]
    cities_nba['NBA'] = cities_nba['NBA'].replace(regex=r'\[.*\]', value='').replace({'—': np.nan, '': np.nan})
    cities_nba.dropna(inplace=True)
    cities_nba['NBA'] = [re.findall(r'[A-Z0-9][^A-Z]*', re.sub(r'\s', '', team)) for team in cities_nba['NBA']]
    
    new_df = []
    for i in cities_nba.index:
        for team in cities_nba['NBA'][i]:
            new_df.append([cities_nba['Metropolitan'][i], team])
    cities_nba_full = pd.DataFrame(new_df, columns=cities_nba.columns)
    
    nba = nba2018.merge(cities_nba_full, left_on='team_short', right_on='NBA')[['Metropolitan', 'W', 'L']]
    nba['WinLoss_NBA'] = nba['W'] / (nba['W']+nba['L'])
    nba = nba.groupby('Metropolitan').agg('mean').reset_index()
    
    
    
    mlb2018 = mlb_df.loc[mlb_df['year'] == 2018, ['team', 'W', 'L']]
    mlb2018['team_short'] = [team.split(' ')[-1] for team in mlb2018['team']]
    mlb2018 = mlb2018.astype({'W': 'int', 'L': 'int'})
    mlb2018 = mlb2018[['team_short', 'W', 'L']]
    mlb2018['team_short'][mlb2018['team_short'].duplicated()] += '1'
    
    cities_mlb = cities[['Metropolitan', 'MLB']]
    cities_mlb['MLB'] = cities_mlb['MLB'].replace(regex=r'\[.*\]', value='').replace({'—': np.nan, '': np.nan})
    cities_mlb.dropna(inplace=True)
    cities_mlb['MLB'] = [re.findall(r'[A-Z][^A-Z]*', re.sub(r'\s', '', team)) for team in cities_mlb['MLB']]
    cities_mlb['MLB'][cities_mlb['Metropolitan'] == 'Chicago'][3][2] = 'Sox1'
    
    new_df = []
    for i in cities_mlb.index:
        for team in cities_mlb['MLB'][i]:
            new_df.append([cities_mlb['Metropolitan'][i], team])
    cities_mlb_full = pd.DataFrame(new_df, columns=cities_mlb.columns)
    
    mlb = mlb2018.merge(cities_mlb_full, left_on='team_short', right_on='MLB')[['Metropolitan', 'W', 'L']]
    mlb['WinLoss_MLB'] = mlb['W'] / (mlb['W']+mlb['L'])
    mlb = mlb.groupby('Metropolitan').agg('mean').reset_index()
    
    
    
    nfl2018 = nfl_df.loc[nfl_df['year'] == 2018, ['team', 'W', 'L']]
    nfl2018 = nfl2018[nfl2018['W'].str.isdigit()]
    nfl2018['team'] = nfl2018['team'].replace(regex=r'[\*\+]*$', value='')
    nfl2018['team_short'] = [team.split(' ')[-1] for team in nfl2018['team']]
    nfl2018 = nfl2018.astype({'W': 'int', 'L': 'int'})
    nfl2018 = nfl2018[['team_short', 'W', 'L']]
    
    cities_nfl = cities[['Metropolitan', 'NFL']]
    cities_nfl['NFL'] = cities_nfl['NFL'].replace(regex=r'\[.*\]', value='').replace({'—': np.nan, '': np.nan})
    cities_nfl.dropna(inplace=True)
    cities_nfl['NFL'] = [re.findall(r'[A-Z0-9][^A-Z]*', re.sub(r'\s', '', team)) for team in cities_nfl['NFL']]
    
    new_df = []
    for i in cities_nfl.index:
        for team in cities_nfl['NFL'][i]:
            new_df.append([cities_nfl['Metropolitan'][i], team])
    cities_nfl_full = pd.DataFrame(new_df, columns=cities_nfl.columns)
    
    nfl = nfl2018.merge(cities_nfl_full, left_on='team_short', right_on='NFL')[['Metropolitan', 'W', 'L']]
    nfl['WinLoss_NFL'] = nfl['W'] / (nfl['W']+nfl['L'])
    nfl = nfl.groupby('Metropolitan').agg('mean').reset_index()
    
    
    
    all_sports = (nhl.merge(nba, on='Metropolitan', how='outer')
                  .merge(mlb, on='Metropolitan', how='outer')
                  .merge(nfl, on='Metropolitan', how='outer'))
    
    
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in range(4):
        for j in range(4):
            sp_i = sports[i]
            sp_j = sports[j]
            p_values.loc[sp_i, sp_j] = stats.ttest_rel(all_sports['WinLoss_'+sp_i], 
                                                       all_sports['WinLoss_'+sp_j], 
                                                       nan_policy='omit')[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values